In [ ]:
#Importing libraries

import numpy as np
from numpy import ndarray
import math

import pandas as pd
from collections import defaultdict
import csv

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import gridspec
from matplotlib import colors as mcolors

from scipy.stats import norm
from scipy.stats import poisson
from scipy.optimize import curve_fit
from scipy.special import factorial

In [ ]:
#Importing data

columns = defaultdict(list)

with open('counting.csv', 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        for (k,v) in row.items():
            columns[k].append(v)
            while '' in columns[k]:
                columns[k].remove('')
                

J1 = list(map(int,columns['E11g']))
J2 = list(map(int,columns['E12g']))
J3 = list(map(int,columns['E13g']))

freq1 = 5
T1 = 1/freq1

tim1 = 5*60

zipped1 = zip(J1,J2,J3)
sumdat1 = [sum(item) for item in zipped1]

d1 = np.array([J1,J2,J3])
leng1 = len(d1)

K1 = list(map(int,columns['E21g']))
K2 = list(map(int,columns['E22g']))
K3 = list(map(int,columns['E23g']))

T2 = 2

zipped2 = zip(K1,K2,K3)
sumdat2 = [sum(item) for item in zipped2]

d2 = np.array([K1,K2,K3])
leng2 = len(d2)

R1 = list(map(int, columns['E31g']))
R2 = list(map(int, columns['E32g']))
R3 = list(map(int, columns['E33g']))
R4 = list(map(int, columns['E34g']))
R5 = list(map(int, columns['E35g']))
R6 = list(map(int, columns['E36g']))
R7 = list(map(int, columns['E37g']))
R8 = list(map(int, columns['E38g']))
R9 = list(map(int, columns['E39g']))
R10 = list(map(int, columns['E310g']))
R11 = list(map(int, columns['E311g']))
R12 = list(map(int, columns['E312g']))
R13 = list(map(int, columns['E313g']))
R14 = list(map(int, columns['E314g']))
R15 = list(map(int, columns['E315g']))
R16 = list(map(int, columns['E316g']))
R17 = list(map(int, columns['E317g']))
R18 = list(map(int, columns['E318g']))
R19 = list(map(int, columns['E319g']))
R20 = list(map(int, columns['E320g']))

freq3 = 10
T3 = 1/freq3

zipped3 = zip(R1,R2,R3,R4,R5,R6,R7,R8,R9,R10,R11,R12,R13,R14,R15,R16,R17,R18,R19,R20)
sumdat3 = [sum(item) for item in zipped3]

d3 = np.array([R1,R2,R3,R4,R5,R6,R7,R8,R9,R10,R11,R12,R13,R14,R15,R16,R17,R18,R19,R20])
leng3 = len(d3)

sumvert = [j for j in list(map(sum, d3))]

H1 = list(map(int,columns['E41g']))
H2 = list(map(int,columns['E42g']))
H3 = list(map(int,columns['E43g']))
H4 = list(map(int,columns['E44g']))
H5 = list(map(int,columns['E45g']))

T4 = 2

zipped4 = zip(H1,H2,H3,H4,H5)
sumdat4 = [sum(item) for item in zipped4]

d4 = np.array([H1,H2,H3,H4,H5])
leng4 = len(d4)


with open('techwed2019.csv', 'r') as g:
    reader = csv.DictReader(g)
    for row in reader:
        for (l,b) in row.items():
            columns[l].append(b)
            while '' in columns[k]:
                columns[l].remove('')

tech = np.array(list(map(int, columns['c'])))

In [ ]:
#Defining functions

def poisson(x, lamb):
    return (lamb**x/factorial(x)) * np.exp(-lamb)

def gauss(x, mu, sig):
    return 1/(sig*np.sqrt(2*np.pi)*np.exp((x-mu)**2/(2*sig**2)))

def chi(func, y):
    return np.sum((y-func)**2/y)

def nozeros(dep, data):
    com = zip(dep, data)
    noz = [(val, key) for (val, key) in com if key != 0]
    l1, l2 = zip(*noz)
    return l1,l2

plt.rcParams.update({'font.size': 12})

def woke(data, NAME, per):
    gs = gridspec.GridSpec(2,1, height_ratios=[3,1])
    plt.figure(figsize=(7,7))
    bins=range(max(data)+1)
    ax0=plt.subplot(gs[0])
    plt.grid(True)
    e = plt.hist(data, bins, facecolor='green', alpha=0.75, rwidth=0.9, label=NAME)
    mean = np.mean(data)
    variance = np.var(data)
    sigma = np.sqrt(variance)
    x = np.linspace(min(data), max(data))
    entries, bin_edges, patches = e
    area = sum(np.diff(bin_edges)*entries)
    bin_middles = 0.5*(bin_edges[1:] + bin_edges[:-1])
    pparameters, pcov_matrix = curve_fit(poisson, bin_middles, entries/area)
    gparameters, gcov_matrix = curve_fit(gauss, bin_middles, entries/area)
    countg = np.abs(gparameters[0]/per)
    countp = np.abs(pparameters[0]/per)
    nzbin, nzent = nozeros(bin_middles, entries)
    chig = chi(area*gauss(nzbin, *gparameters), nzent)
    chip = chi(area*poisson(nzbin, *pparameters), nzent)
    plt.ylabel('Occurrence')
    plt.plot(x, area*poisson(x, *pparameters), 'r--', lw=2, label='Poissonian fit')
    plt.plot(x, area*gauss(x, *gparameters), 'b--', lw=2, label='Gaussian fit')
    plt.errorbar(bin_middles, entries, yerr=np.sqrt(entries), fmt='k.')
    ax1=plt.subplot(gs[1],sharex=ax0)
    plt.xlim((min(data), max(data)))
    plt.plot(x,0*x, 'k', lw=1)
    plt.errorbar(bin_middles, entries-area*gauss(bin_middles,*gparameters), yerr=np.sqrt(entries), fmt='b.', label='Residuals for Gaussian fit')
    plt.errorbar(bin_middles, entries-area*poisson(bin_middles,*pparameters), yerr=np.sqrt(entries), fmt='r.', label='Residuals for Poisson fit')
    ax0.legend(loc='upper right')
    ax1.legend(loc='upper right')
    plt.ylabel('Standardized residuals')
    plt.xlabel('Counts per time bin')
    plt.grid(True)
    plt.show()    
    print('Count rate Gauss')
    print(countg)
    print('Error:')
    print(4*gparameters[1]/np.sqrt(len(data)))
    print('Count rate Poisson')
    print(countp)
    print('Error:')
    print(4*np.sqrt(pparameters[0])/np.sqrt(len(data)))
    print('Chi squared Gauss:')
    print(chig)
    print('Chi squared Poisson')
    print(chip)
    return countg, countp

# EXP. 1

In [ ]:
avg11 = leng1*[0]
avg12 = leng1*[0]
for i in range(leng1):
    avg11[i], avg12[i] = woke(d1[i], 'Experiment 2, run %s' %(i+1), T1)
    
print('Summary of all runs:')
print('Mean count Gauss:')
print(np.mean(avg11))
print('Error:')
print(np.std(avg11)/np.sqrt(len(avg11)))
print('Mean count Poisson:')
print(np.mean(avg12))
print('Error:')
print(np.std(avg12)/np.sqrt(len(avg12)))

In [ ]:
#Sum of all runs

woke(sumdat1, 'Experiment 3, sum of runs', T1*leng1)

# EXP. 2

In [ ]:
avg21 = leng2*[0]
avg22 = leng2*[0]
for i in range(leng2):
    avg21[i], avg22[i] = woke(d2[i], 'Experiment 2, run %s' %(i+1), T2)
    
print('Summary of all runs:')
print('Mean count Gauss:')
print(np.mean(avg21))
print('Error:')
print(np.std(avg21)/np.sqrt(len(avg21)))
print('Mean count Poisson:')
print(np.mean(avg22))
print('Error:')
print(np.std(avg22)/np.sqrt(len(avg22)))

In [ ]:
#Sum of all runs

woke(sumdat2, 'Experiment 2, sum of runs', T2*leng2)

# EXP. 4

In [ ]:
avg41 = leng4*[0]
avg42 = leng4*[0]
for i in range(leng4):
    avg41[i], avg42[i] = woke(d4[i], 'Experiment 1, run %s' %(i+1), 2)
    
print('Summary of all runs')
print('Mean count Gauss:')
print(np.mean(avg41))
print('Error:')
print(np.std(avg41)/np.sqrt(len(avg41)))
print('Mean count Poisson:')
print(np.mean(avg42))
print('Error:')
print(np.std(avg42)/np.sqrt(len(avg42)))

In [ ]:
#Sum of all runs

woke(sumdat4, 'Experiment 4, sum of runs', T4*leng4)

# EXP. 3

In [ ]:
avg31 = leng3*[0]
avg32 = leng3*[0]
for i in range(leng3):
    avg31[i], avg32[i] = woke(d3[i], 'Experiment 3, run %s' %(i+1), T3)

print('Summary of all runs:')
print('Mean count Gauss:')
print(np.mean(avg31))
print('Error:')
print(np.std(avg31)/np.sqrt(len(avg31)))
print('Mean count Poisson:')
print(np.mean(avg32))
print('Error:')
print(np.std(avg32)/np.sqrt(len(avg32)))

In [ ]:
#Sum of all runs

woke(sumdat3, 'Experiment 3, sum of runs', T3*leng3)

In [ ]:
#Vertical sum of all runs

gs = gridspec.GridSpec(2,1, height_ratios=[3,1])
plt.figure(figsize=(7,7))

bins=6

ax0=plt.subplot(gs[0])
plt.grid(True)
e = plt.hist(sumvert, bins, facecolor='green', alpha=0.75, rwidth=0.9, label='Experiment 3, vertical sum of runs')
mean = np.mean(sumvert)
count = mean/2
variance = np.var(sumvert)
sigma = np.sqrt(variance)
x = np.linspace(min(sumvert), max(sumvert))
entries, bin_edges, patches = e
area = sum(np.diff(bin_edges)*entries)
bin_middles = 0.5*(bin_edges[1:] + bin_edges[:-1])
gparameters, gcov_matrix = curve_fit(gauss, bin_middles, entries/area)
chig = chi(area*gauss(bin_middles, *gparameters), entries)
plt.ylabel('Occurrence')
plt.plot(x, area*gauss(x,mean,sigma), 'b--', lw=2, label='Gaussian fit')
plt.errorbar(bin_middles, entries, yerr=np.sqrt(entries), fmt='k.')
ax1=plt.subplot(gs[1],sharex=ax0)
plt.xlim((min(sumvert), max(sumvert)))
plt.plot(x,0*x, 'k', lw=1)
plt.errorbar(bin_middles, entries-area*gauss(bin_middles,mean,sigma), yerr=0, fmt='b.', label='Residuals for Gaussian fit')
ax0.legend(loc='upper right')
ax1.legend(loc='upper right')
plt.ylabel('Standardized residuals')
plt.xlabel('Counts per time bin (divided by 100)')
plt.grid(True)
plt.show()

print('Chi squared test for Gaussian fit:')
print(chig)
print('Count rate:')
print(count)

#Poisson distribution could not be plotted due to the high x-values
#Specifically, we get overflow errors because of the factorial in the Poisson distribution

# TECH. ANALYSIS

In [ ]:
tech2=[]
tech5=[]
tech20=[]

i=0
while i<len(tech):
    tech2=np.append(tech2,np.sum(tech[i:i+1]))
    i+=2
i=0
while i<len(tech):
    tech5=np.append(tech5,np.sum(tech[i:i+4]))
    i+=5
i=0
while i<len(tech):
    tech20=np.append(tech20,np.sum(tech[i:i+19]))
    i+=20

In [ ]:
woke(tech, 'Technician data', T1)
woke(list(map(int,tech2)), 'Technician data, partitioned in half', T1*2)
woke(list(map(int,tech5)), 'Technician data, partitioned in fifths', T1*5)

In [ ]:
ntech20 = list(map(int,tech20))
period = T1*20

gs = gridspec.GridSpec(2,1, height_ratios=[3,1])
plt.figure(figsize=(7,7))
bins=range(max(ntech20)+1)
ax0=plt.subplot(gs[0])
plt.grid(True)
e = plt.hist(ntech20, bins, facecolor='green', alpha=0.75, rwidth=0.9, label='Technician data, partitioned in twentieths')
mean = np.mean(ntech20)
variance = np.var(ntech20)
sigma = np.sqrt(variance)
x = np.linspace(min(ntech20), max(ntech20))
entries, bin_edges, patches = e
area = sum(np.diff(bin_edges)*entries)
bin_middles = 0.5*(bin_edges[1:] + bin_edges[:-1])
pparameters, pcov_matrix = curve_fit(poisson, bin_middles, entries/area)
gparameters, gcov_matrix = curve_fit(gauss, bin_middles, entries/area)
countg = np.abs(gparameters[0]/period)
countp = np.abs(pparameters[0]/period)
nzbin, nzent = nozeros(bin_middles, entries)
chig = chi(area*gauss(nzbin, *gparameters), nzent)
chip = chi(area*poisson(nzbin, *pparameters), nzent)
plt.ylabel('Occurrence')
plt.plot(x, area*poisson(x, *pparameters), 'r--', lw=2, label='Poissonian fit')
plt.plot(x, area*gauss(x,mean,sigma), 'b--', lw=2, label='Gaussian fit')
plt.errorbar(bin_middles, entries, yerr=np.sqrt(entries), fmt='k.')
ax1=plt.subplot(gs[1],sharex=ax0)
plt.xlim((min(ntech20), max(ntech20)))
plt.plot(x,0*x, 'k', lw=1)
plt.errorbar(bin_middles, entries-area*gauss(bin_middles, mean, sigma), yerr=np.sqrt(entries), fmt='b.', label='Residuals for Gaussian fit')
plt.errorbar(bin_middles, entries-area*poisson(bin_middles,*pparameters), yerr=np.sqrt(entries), fmt='r.', label='Residuals for Poisson fit')
ax0.legend(loc='upper right')
ax1.legend(loc='upper right')
plt.ylabel('Standardized residuals')
plt.xlabel('Counts per time bin')
plt.grid(True)
plt.show()

print('Count rate Gauss')
print(countg)
print('Error:')
print(10*gparameters[1]/np.sqrt(len(ntech20)))
print('Count rate Poisson')
print(countp)
print('Error:')
print(10*np.sqrt(pparameters[0])/np.sqrt(len(ntech20)))

# COUNTING DATA

In [ ]:
N = 3

p_means = (np.mean(avg12), np.mean(avg22), np.mean(avg32))
p_std = (1.2, 1.2, 1.2)

g_means = (np.mean(avg11), np.mean(avg21), np.mean(avg31))
g_std = (1, 0.5, 1)

gcum_means = (12.33, 11.39, 11.79)
gcum_std = (1,0.5,1)

ind = np.arange(N)  # the x locations for the groups
width = 0.2       # the width of the bars

fig, ax = plt.subplots()
plt.ylim((10.8, 18))
rects1 = ax.bar(ind, p_means, width, color='r', yerr=p_std)
rects2 = ax.bar(ind + width, g_means, width, color='y', yerr=g_std)
rects3 = ax.bar(ind + 2*width, gcum_means, width, color='b', yerr=gcum_std)

# add some text for labels, title and axes ticks
ax.set_ylabel('Count rate (Bq)')
ax.set_title('')
ax.set_xticks(ind + width)
ax.set_xticklabels(('E1', 'E2', 'E3'))

ax.legend((rects1[0], rects2[0], rects3[0]), ('Average Gauss count rate of individual runs', 'Average Poisson count rate of individual runs','Gauss cumulative'), loc='best')


def autolabel(rects):
    """
    Attach a text label above each bar displaying its height
    """
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2., 1.05*height,
                '%d' % int(height),
                ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)
autolabel(rects3)

plt.show()